In [7]:
import numpy as np
from fractions import gcd

In [13]:
def psi(a,b):
    temp = int(np.floor(a/b))
    r = a-temp*b
    return r

In [37]:
def LHNF(H,A):
    LH = np.transpose(H)
    U = np.matmul(H,np.linalg.inv(A))
    return  (LH, U)

In [122]:
def HNF(A, V):
    """
    Args:
        A (list): a 3x3 input matrix.
        V (list): a 3x3 input matrix.
        
    Returns:
        A (list): a 3x3 input matrix.
        V (list): a 3x3 input matrix.
    """
    A = np.array(A)
    V = np.array(V)
    for t in range(3):
        r = -1
        for s in range(3):
            if A[s][r+1]!=0 or A[s][t]!=0:
                r += 1
                ir = s
                if t== r and A[s][t] < 0:
                    V[:,t] = np.sign(A[s][t])*V[:,t]
                    A[:,t] = np.sign(A[s][t])*A[:,t]
                else:
                    A,V = row_one_gcd(A,V,s,r,t)
                for l in range(r-1):
                    V[:,l] = V[:,l] - psi(A[s][l],A[s][r])*V[:,r]
                    A[:,l] = A[:,l] - psi(A[s][l],A[s][r])*A[:,r]
                if t==r:
                    break
    return (A,V)    

In [123]:
def row_one_gcd(A,V,i,j,l):
    from fractions import gcd
    if A[i][j] != 0 and A[i][l] !=0 and j!=l:
        d = gcd(A[i][j],A[i][l])
        uv_found = False
        u=0
        v=0
        while not uv_found:
            v = (d-u*A[i][j])/float(A[i][l])
            if v%1 == 0:
                v = int(v)
                uv_found = True
            else: 
                u += 1
        temp = np.transpose(np.dot(np.transpose([V[:,j],V[:,l]]),[[u, -A[i][l]/d],[v, A[i][j]/d]]))
        V[:,j] = temp[0]
        V[:,l] = temp[1]
        temp = np.transpose(np.dot(np.transpose([A[:,j],A[:,l]]),[[u, -A[i][l]/d],[v, A[i][j]/d]]))
        A[:,j] = temp[0]
        A[:,l] = temp[1]
    return (A,V)

In [159]:
def diagtosmith(A,U,V):
    """
    Args:
        A (list): a 3x3 input matrix.
        U (list): a vector of 3 elements.
        V (list): a vector of 3 elements.    
        
    Returns:
        A (list): a 3x3 input matrix.
        U (list): a 3x3 input matrix.
        V (list): a 3x3 input matrix.
    """
    U = np.array(U)
    A = np.array(A)
    V = np.array(V)
    for k in range(2):
        for l in reversed(range(k,2)):
            if not A[l+1][l+1]%A[l][l]==0:
                g = A[l][l]*A[l+1][l+1]
                A[l][l]=gcd(A[l][l],A[l+1][l+1])
                A[l+1][l+1] = g/A[l][l]
                d = gcd(A[l][l],A[l+1][l+1])
                uv_found = False
                u=0
                v=0
                while not uv_found:
                    v = (d-u*A[l][l])/float(A[l+1][l+1])
                    if v%1 == 0:
                        uv_found = True
                    else: 
                        u += 1
                un = np.matmul([[u,v],[-A[l+1][l+1]/d, A[l][l]/d]],[U[:,l],U[:,l+1]])
                U[:,l] = un[0]
                U[:,l+1] = un[1]
                vn = np.transpose(np.matmul(np.transpose([V[:,l],V[:,l+1]]),[[1,-v*A[l+1][l+1]/d],[1, u*A[l][l]/d]]))
                V[:,l] = vn[0]
                V[:,l+1] = vn[1]
    for l in range(3):
        if A[l][l] !=0:
            beta = np.sign(A[l][l])
            V[:,l] = V[:,l]*beta
            A[l][l] = A[l][l]*beta
    return (A,U,V)

In [180]:
def SNF(A):
    U = [[1,0,0],[0,1,0],[0,0,1]]
    V = [[1,0,0],[0,1,0],[0,0,1]]
    count = 0
    while not np.allclose([A[0][1],A[0][2],A[1][2],A[1][0],A[2][0],A[2][1]],0):
        print("count",count)
        A, V = HNF(A, V)
        A, U = LHNF(A, U)
        count += 1
    #A, U, V = diagtosmith(A,U,V)
    return A, U, V

In [190]:
A = [[63,0,0],[0,1,0],[0,432,1175]]
print(np.linalg.det(A))
S,U,V = SNF(A)
S

74025.0
('count', 0)
('count', 1)


array([[  63,    0,    0],
       [   0,    1,    0],
       [   0,    0, 1175]])

In [189]:
np.round(np.matmul(U,np.matmul(A,V)),1).astype(int)

array([[  63,    0,    0],
       [   0,    1, -432],
       [   0,    0, 1175]])

In [170]:
np.round(U).astype(int)

array([[       1,      -63,  4663575],
       [       0,        1,   -74025],
       [       0,     -432, 31978801]])

In [179]:
HNF(A,[[1,0,0],[0,1,0],[0,0,1]])

(array([[  63,    0,    0],
        [   0,    1,    0],
        [   0,  432, 1175]]), array([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]]))

In [191]:
temp = [[3,      0,      0],
        [0,      1,      0],
        [717,    707,    830]]

In [344]:
def get_min_val(N):
    """Finds the minimum value of a matrix that's not in a row or
    column of zeros.
    
    Args:
        N (list): The input matrix.
    Returns:
        The minimum value in the matrix that isn't already in
        a zeroed row or column."""
    
    temp_N = abs(N)
    list_N = sorted(temp_N.flatten())
    for v in list_N:
        if v == 0:
            continue
        row, col = np.argwhere(temp_N==v)[0]
        if (list(temp_N[:,col]).count(0) < 2 or 
            list(temp_N[row,:]).count(0) < 2):
            return N[row,col], row, col
        else:
            temp_N[row,col] = -1
    print("N",N)
    print("temp_N",temp_N)

In [355]:
def Rods_way(N):
    """Finds the SmithNormalForm by reducing starting with the 
    smallest entry in the matrix.
    
    Args:
        N (list): An integer 3x3 matrix.
        
    Returns:
        L,S,R: the left transform, the SNF and the right 
        transform.
    """
    from copy import deepcopy
    S = np.array(N)
    L = np.identity(3)
    R = np.identity(3)
    
    is_snf = False
    cur_diag = 0
    count = 0
    while not is_snf and count < 100:
        min_val, row, col = get_min_val(S)
        #First reduce the row
        for j in range(3):
            if j == col:
                continue
            multiple = int(S[row,j]/min_val)
            if multiple==0:
                continue
            S[:,j] = S[:,j]-multiple*S[:,col]
            R[:,j] = R[:,j]-multiple*R[:,col]
            
        #then reduce the column
        for j in range(3):
            if j == row:
                continue
            multiple = int(S[j,col]/min_val)
            if multiple==0:
                continue
            S[j,:] = S[j,:]-multiple*S[row,:]
            L[j,:] = L[j,:]-multiple*L[row,:]

        if (np.allclose(S[cur_diag:,cur_diag:]%min_val,0)):
            if cur_diag != col:
                #Swap rows and columns
                tmp_col = deepcopy(S[:,cur_diag])
                S[:,cur_diag] = deepcopy(S[:,col])
                S[:,col] = tmp_col
                tmp_col = deepcopy(R[:,cur_diag])
                R[:,cur_diag] = deepcopy(R[:,col]) 
                R[:,col] = tmp_col
            if cur_diag != row:
                tmp_row = deepcopy(S[cur_diag,:])
                S[cur_diag,:] = deepcopy(S[row,:])
                S[row,:] = tmp_row
                tmp_row = deepcopy(L[cur_diag,:])
                L[cur_diag,:] = deepcopy(L[row,:])
                L[row,:] = tmp_row     
            cur_diag += 1

        count += 1
        if ((S[0][1]==0 and S[0][2]==0 and S[1][0]==0 and S[1][2]==0
            and S[2][0]==0 and S[2][1]==0)  and S[2][2]%S[1][1]==0 
            and S[1][1]%S[0][0]==0):
            is_snf = True
        if (cur_diag==0 and row == 0 and col == 0 and
            list(S[0,:]).count(0)==2 and list(S[:,0]).count(0)==2 and
            not sp.allclose(S%min_val,0)):
                mrow, mcol = np.argwhere(S==S.max())
                S[0,:] = S[0, :] + S[mrow,:]
                L[0,:] = L[0, :] + L[mrow,:]
        if ((cur_diag==1) and
            list(S[1,:]).count(0)==2 and list(S[:,1]).count(0)==2 and
            not np.allclose(S[1:,1:]%min_val,0)):
                S[1,:] = S[1, :] + S[2,:]
                L[1,:] = L[1, :] + L[2,:]
        for j in range(3):
            if S[j,j] < 0:
                S[j,:] = -S[j,:]
                L[j,:] = -L[j,:]
    if count == 100:
        print("Failed to find SNF in 100 iterations.")
    if not np.allclose(np.matmul(np.matmul(L,N),R),S):
        print("Transformation failed in SNF.")
            
    return L, S, R

In [356]:
np.set_printoptions(suppress=True)
N = np.array([[3,0,0],[0,1,0],[717,707,830]])
L,S,R = Rods_way(N)
print(L)
print(S)
print(R)

[[      0.       1.       0.]
 [   -238.    -707.       1.]
 [-197779. -587517.     831.]]
[[   1    0    0]
 [   0    1    0]
 [   0    0 2490]]
[[   0.  277. -830.]
 [   1.    0.    0.]
 [   0.   -1.    3.]]


In [353]:
np.matmul(np.matmul(L,N),R)


array([[    1.,     0.,     0.],
       [    0.,     1.,     0.],
       [    0.,     0.,  2910.]])

In [247]:
sorted(abs(N).flatten())

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [249]:
a = [0,1,0]
a.count(0)

2

In [273]:
np.linalg.det(N)

-3.0000000000000018

In [275]:
np.allclose(N%1,0)

True

In [278]:
N[1:,1:]

array([[5, 6],
       [8, 9]])

In [347]:
N.max()

950

In [357]:
from os import getcwd
getcwd()

'/Users/wileymorgan/codes/opf_kgrids/support/notebooks'